# 第22章 动态属性和特性

- 数据属性和方法统称为属性。方法是可调用的属性。
- 动态属性的接口和数据属性一样，按需计算。
- 用户定义的类通过`__getattr__`方法可以实现动态属性。

## 22.1 使用动态属性转换数据

### 22.1.1 FrozenJson第1版

In [1]:
from collections import abc


class FrozenJSON_V1:
    """一个只读接口，该接口使用属性表示法访问JSON类对象"""

    def __init__(self, mapping):
        # 私有属性，构建字典
        self.__data = dict(mapping)

    def __getattr__(self, name):
        """当未指定名称的属性，调用该方法"""
        try:
            # 如果能匹配到字典的某个值，就返回对应的属性
            return getattr(self.__data, name)  
        except AttributeError:
            # 否则，从字典中获取name对应的项，返回调用build方法得到的结果
            return self.__class__.build(self.__data[name])  # <4>

    def __dir__(self):
        # 支持在Python控制台进行自动补全
        return self.__data.keys()

    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            # 如果是一个map，则构建一个FrozenJSON对象，大鹅类型
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            # 如果是列表，将obj的每一项进行递归构建列表
            return [cls.build(item) for item in obj]
        else:
            return obj

In [2]:
import json
raw_feed = json.load(open('./data/osconfeed.json'))
feed = FrozenJSON_V1(raw_feed)

In [3]:
len(feed.Schedule.speakers)

357

In [4]:
feed.keys()

dict_keys(['Schedule'])

In [5]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [6]:
for key, value in sorted(feed.Schedule.items()):
    print(f'{len(value):3} {key}')

  1 conferences
484 events
357 speakers
 53 venues


### 22.1.2 FrozenJSON第2版

修改`__init__`方法，解决与Python关键字同名的属性名问题。

In [7]:
from collections import abc
import keyword

class FrozenJSON_V2:
    """一个只读接口，该接口使用属性表示法访问JSON类对象"""

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value

    def __getattr__(self, name):
        """当未指定名称的属性，调用该方法"""
        try:
            # 如果能匹配到字典的某个值，就返回对应的属性
            return getattr(self.__data, name)  
        except AttributeError:
            # 否则，从字典中获取name对应的项，返回调用build方法得到的结果
            return self.__class__.build(self.__data[name])  # <4>

    def __dir__(self):
        # 支持在Python控制台进行自动补全
        return self.__data.keys()

    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            # 如果是一个map，则构建一个FrozenJSON对象，大鹅类型
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            # 如果是列表，将obj的每一项进行递归构建列表
            return [cls.build(item) for item in obj]
        else:
            return obj

In [8]:
student = FrozenJSON_V2({'name': 'Jim Bo', 'class': 1982})

### 22.1.3 FrozenJSON第3版

使用`__new__`代替`build`方法。

In [9]:
from collections import abc
import keyword

class FrozenJSON_V3:
    """一个只读接口，该接口使用属性表示法访问JSON类对象"""
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            # 如果是一个map，则构建一个FrozenJSON对象，大鹅类型
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            # 如果是列表，将obj的每一项进行递归构建列表
            return [arg(item) for item in arg]
        else:
            return arg 
    
    
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value

    def __getattr__(self, name):
        """当未指定名称的属性，调用该方法"""
        try:
            # 如果能匹配到字典的某个值，就返回对应的属性
            return getattr(self.__data, name)  
        except AttributeError:
            # 否则，从字典中获取name对应的项，返回调用build方法得到的结果
            return self.__class__(self.__data[name])  # <4>

    def __dir__(self):
        # 支持在Python控制台进行自动补全
        return self.__data.keys()

## 22.2 计算特性

### 22.2.1 第1步：数据驱动属性创建

目标：通过speakers的编号获取对应的数据。

In [10]:
import json

JSON_PATH = './data/osconfeed.json'

class Record:
    def __init__(self, **kwargs):
        # 根据关键字参数构建带属性的实例
        self.__dict__.update(kwargs)  

    def __repr__(self):
        return f'<{self.__class__.__name__} serial={self.serial!r}>'  # <2>

def load(path=JSON_PATH):
    records = {}  
    # 解析JSON字符串
    with open(path) as fp:
        raw_data = json.load(fp)  
    # 迭代4个顶级列表    
    for collection, raw_records in raw_data['Schedule'].items():
        record_type = collection[:-1]  
        for raw_record in raw_records:
            key = f'{record_type}.{raw_record["serial"]}' 
            records[key] = Record(**raw_record)  
    return records

In [11]:
records = load(JSON_PATH)

In [12]:
speaker = records['speaker.3471']
speaker

<Record serial=3471>

In [13]:
speaker.name, speaker.twitter

('Anna Martelli Ravenscroft', 'annaraven')

### 22.2.2 第2步：通过特性获取链接的记录

目标：给定一个`event`记录，读取`venue`特性得到一个`Record`对象。

In [15]:
import inspect  # <1>
import json

JSON_PATH = './data/osconfeed.json'

class Record:
    # 存放对load函数返回的字典的引用
    __index = None

    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __repr__(self):
        return f'<{self.__class__.__name__} serial={self.serial!r}>'

    @staticmethod 
    def fetch(key):
        if Record.__index is None:
            # 如果没有键，则从解析加载
            Record.__index = load()
        return Record.__index[key]

In [16]:
class Event(Record):

    def __repr__(self):
        try:
            # 如果实例有name属性，则设置字符串表示形式
            return f'<{self.__class__.__name__} {self.name!r}>'  # <2>
        except AttributeError:
            return super().__repr__()

    @property
    def venue(self):
        # 根据venue_serial属性构建一个Key，传给fetch方法
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key)

In [17]:
def load(path=JSON_PATH):
    records = {}
    with open(path) as fp:
        raw_data = json.load(fp)
    for collection, raw_records in raw_data['Schedule'].items():
        # 删除尾部s
        record_type = collection[:-1]
        # 首字母大写，用于适配类名
        cls_name = record_type.capitalize()  
        cls = globals().get(cls_name, Record)  
        if inspect.isclass(cls) and issubclass(cls, Record):
            factory = cls
        else:
            factory = Record
        for raw_record in raw_records:
            key = f'{record_type}.{raw_record["serial"]}'
            records[key] = factory(**raw_record)
    return records

In [18]:
event = Record.fetch('event.33950')
event

<Event 'There *Will* Be Bugs'>

In [19]:
event.venue

<Record serial=1449>

In [20]:
event.venue.name

'Portland 251'

In [21]:
event.venue_serial

1449

### 22.2.3 第3步：用特性覆盖现有属性

In [22]:
class Event(Record):

    def __repr__(self):
        try:
            return f'<{self.__class__.__name__} {self.name!r}>'
        except AttributeError:
            return super().__repr__()

    @property
    def venue(self):
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key)

    @property
    def speakers(self):
        spkr_serials = self.__dict__['speakers']
        fetch = self.__class__.fetch
        # 获取键与spkr_serials中的数值匹配的所有记录，构成一个列表
        return [fetch(f'speaker.{key}')
                for key in spkr_serials]

- 直接通过对象的`__dict__`属性读写数据，是Python元编程常见的技巧。

### 22.2.4 第4步：自己实现特性缓存

In [23]:
class Event(Record):

    def __init__(self, **kwargs):
        # 禁用键共享优化
        self.__speaker_objs = None
        super().__init__(**kwargs)

    def __repr__(self):
        try:
            return f'<{self.__class__.__name__} {self.name!r}>'
        except AttributeError:
            return super().__repr__()

    @property
    def venue(self):
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key)

    @property
    def speakers(self):
        # 实现缓存
        if self.__speaker_objs is None:
            spkr_serials = self.__dict__['speakers']
            fetch = self.__class__.fetch
            self.__speaker_objs = [fetch(f'speaker.{key}')
                    for key in spkr_serials]
        return self.__speaker_objs

### 22.2.5 第5步：使用functools缓存特性

- `functools.cached_property`装饰器把方法的结果缓存在同名实例属性中。
- `@cached_property`装饰器不创建完整的特性，而是创建一个非覆盖型描述符。
- `cached_property()`允许属性写入。
- `@cached_property`装饰器仅在执行查找且不村子啊同名属性时允许，一旦允许，就会写入同名属性。
- 缓存的值可以通过删除属性清空。

In [25]:
from functools import cached_property, cache

class Event(Record):

    def __repr__(self):
        try:
            return f'<{self.__class__.__name__} {self.name!r}>'
        except AttributeError:
            return super().__repr__()

    @cached_property
    def venue(self):
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key)
    
    @property  
    @cache
    def speakers(self):
        spkr_serials = self.__dict__['speakers']
        fetch = self.__class__.fetch
        return [fetch(f'speaker.{key}')
                for key in spkr_serials]

## 22.3 使用特性验证属性

需求：假设有一个销售散装有机食物的电商应用程序，客户可以按量订购坚果、干果或杂粮。在这个系统中，每个订单中有一系列的商品。

### 22.3.1 LineItem类第1版：表示订单中商品的类

In [26]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [27]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.subtotal()

69.5

In [28]:
raisins.weight = -20
raisins.subtotal()

-139.0

### 22.3.2 LineItem类第2版：能验证值的特性

问题：数据可能会被设置为负值，引发无效输出。

In [29]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight 
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    @property  
    def weight(self):  
        return self.__weight  

    @weight.setter 
    def weight(self, value):
        if value > 0:
            self.__weight = value  
        else:
            raise ValueError('value must be > 0')  # <7>

## 22.4 定义一个特性工厂函数

目标：保护LineItem对象的weight属性和price属性，只允许设为大于0的值，但是不用手动实现两对几乎一样的读值和设值方法。

In [32]:
def quantity(storage_name):
    """特性工厂函数
    storage_name: 确定各个特性的数据存储在哪里
    """
    def qty_getter(instance):
        return instance.__dict__[storage_name]

    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
    # 构建一个自定义的特性对象
    return property(qty_getter, qty_setter)

In [33]:
class LineItem:
    # 定义为类属性
    weight = quantity('weight')  
    price = quantity('price') 

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight  
        self.price = price

    def subtotal(self):
        return self.weight * self.price  

In [34]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
nutmeg.weight, nutmeg.price

(8, 13.95)

In [35]:
nutmeg.__dict__

{'description': 'Moluccan nutmeg', 'weight': 8, 'price': 13.95}

## 22.5 处理属性的重要属性和函数

### 22.5.1 影响属性处理方式的特殊属性

- `__class__`：对象所属类的引用。Python的某些特殊方法，只在对象的类中而不在实例中寻找。
- `__dict__`：存储对象或类的可写属性的映射。在任何时候都能随意设置新属性。
- `__slots__`：类可以定义这个属性，节省内存。

### 22.5.2 处理属性的内置函数

- `dir([object])`：列出对象的大多数属性。`dir`函数不列出`__dict__`属性本书，但会列出其中的键。
- `getattr(object, name[, default])`：从`object`对象中获取`name`字符串对应的属性。主要用于获取事先不知道名称的属性。属性可能来自对象所属的类或超类。
- `hasattr(object, name)`：如果`object`对象中存在指定的属性，或者能以某种方式（例如继承）通过`object`对象获取指定的属性，则返回`True`。
- `setattr(object, name, value)`：把`object`对象指定属性的值设为`value`。
- `vars([object])`：返回`object`对象的`__dict__`属性。

### 22.5.3 处理属性的特殊方法

- `__delattr__(self, name)`：只要使用`del`语句删除属性，就会调用这个方法。
- `__dir__(self)`：在对象上调用`dir`函数时，会调用这个方法。
- `__getattr__(self, name)`：仅当获取指定的属性失败，搜索过`obj`、`Class`及其超类之后，会调用这个方法。
- `__getattribute__(self, name)`：尝试直接获取指定名称的属性时，始终调用这个方法。
- `__setattr__(self, name, value)`：尝试设置指定名称的属性时，会调用这个方法。

## 22.6 杂谈

讨论统一访问原则：
- 在面向对象编程语言中，是否遵守统一访问原则通常体现在句法上：究竟时读取公开的数据属性，还是调用读值和设值方法。
- Smalltalk和Ruby语言，根本不支持公开的数据属性。所有实例属性都是私有的，必须通过方法来存取。
- Java让程序员在4种访问级别修饰符中选择。

类与函数：
- Python还有一处体现了统一访问原则，函数调用和对象实例化使用相同的句法。
- 构造方法替换成工厂函数的理由：
    1. 通过返回之前构建的实例，限制实例的数量。
    2. 缓存构建过程开销大的对象。
- 构造工厂方法，API的设计者必须提前决定。